In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import re

In [2]:
df = pd.read_csv('../data/synthetic_logs.csv')
df

timestamp           source  \
0     2025-06-27 07:20:25        ModernCRM   
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
3     2025-07-12 00:24:16         ModernHR   
4     2025-06-02 18:25:23    BillingSystem   
...                   ...              ...   
2405  2025-08-13 07:29:25         ModernHR   
2406       1/11/2025 5:32         ModernHR   
2407  2025-08-03 03:07:47    ThirdPartyAPI   
2408     11/11/2025 11:52    BillingSystem   
2409     12/25/2025 13:21  AnalyticsEngine   

                                            log_message    target_label  \
0     nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1        Email service experiencing issues with sending  Critical Error   
2             Unauthorized access to data was attempted  Security Alert   
3     nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4     nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   
...                                                 ...             ...   
2405  nova.osapi_compute.wsgi.server [req-96c3ec98-2...     HTTP Status   
2406  User 3844 account experienced multiple failed ...  Security Alert   
2407  nova.metadata.wsgi.server [req-b6d4a270-accb-4...     HTTP Status   
2408      Email service affected by failed transmission  Critical Error   
2409  Repeated failed login attempts occurred for us...  Security Alert   

     complexity  
0          bert  
1          bert  
2          bert  
3          bert  
4          bert  
...         ...  
2405       bert  
2406       bert  
2407       bert  
2408       bert  
2409       bert  

[2410 rows x 5 columns]

In [3]:
# Extract the messages column
messages = df['log_message']

# Generate embeddings using SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(messages.tolist())

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters = dbscan.fit_predict(embeddings)

# Add the cluster labels to the dataframe
df['cluster'] = clusters
df

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

timestamp           source  \
0     2025-06-27 07:20:25        ModernCRM   
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
3     2025-07-12 00:24:16         ModernHR   
4     2025-06-02 18:25:23    BillingSystem   
...                   ...              ...   
2405  2025-08-13 07:29:25         ModernHR   
2406       1/11/2025 5:32         ModernHR   
2407  2025-08-03 03:07:47    ThirdPartyAPI   
2408     11/11/2025 11:52    BillingSystem   
2409     12/25/2025 13:21  AnalyticsEngine   

                                            log_message    target_label  \
0     nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1        Email service experiencing issues with sending  Critical Error   
2             Unauthorized access to data was attempted  Security Alert   
3     nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4     nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   
...                                                 ...             ...   
2405  nova.osapi_compute.wsgi.server [req-96c3ec98-2...     HTTP Status   
2406  User 3844 account experienced multiple failed ...  Security Alert   
2407  nova.metadata.wsgi.server [req-b6d4a270-accb-4...     HTTP Status   
2408      Email service affected by failed transmission  Critical Error   
2409  Repeated failed login attempts occurred for us...  Security Alert   

     complexity  cluster  
0          bert        0  
1          bert        1  
2          bert        2  
3          bert        0  
4          bert        0  
...         ...      ...  
2405       bert        0  
2406       bert        7  
2407       bert        0  
2408       bert        1  
2409       bert        7  

[2410 rows x 6 columns]

In [5]:
# Group by cluster and filter clusters with more than 10 messages
cluster_counts = df['cluster'].value_counts()
clusters_with_more_than_10 = cluster_counts[cluster_counts > 10].index

# Print the clusters and their corresponding log messages
for cluster in clusters_with_more_than_10:
    print(f"Cluster {cluster}:")
    print(df[df['cluster'] == cluster]['log_message'].tolist())
    print("\n")

Cluster 0:
['nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118', 'nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180', 'nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791', 'nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cd

In [26]:
def classify_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, category in regex_patterns.items():
        if re.search(pattern, log_message, re.IGNORECASE):
            return category
    return "Unknown"

In [27]:
classify_regex("this is message will return NONE.")

'Unknown'

In [28]:
df['regex_label'] = df['log_message'].apply(classify_regex)
df

timestamp           source  \
0     2025-06-27 07:20:25        ModernCRM   
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
3     2025-07-12 00:24:16         ModernHR   
4     2025-06-02 18:25:23    BillingSystem   
...                   ...              ...   
2405  2025-08-13 07:29:25         ModernHR   
2406       1/11/2025 5:32         ModernHR   
2407  2025-08-03 03:07:47    ThirdPartyAPI   
2408     11/11/2025 11:52    BillingSystem   
2409     12/25/2025 13:21  AnalyticsEngine   

                                            log_message    target_label  \
0     nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1        Email service experiencing issues with sending  Critical Error   
2             Unauthorized access to data was attempted  Security Alert   
3     nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4     nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   
...                                                 ...             ...   
2405  nova.osapi_compute.wsgi.server [req-96c3ec98-2...     HTTP Status   
2406  User 3844 account experienced multiple failed ...  Security Alert   
2407  nova.metadata.wsgi.server [req-b6d4a270-accb-4...     HTTP Status   
2408      Email service affected by failed transmission  Critical Error   
2409  Repeated failed login attempts occurred for us...  Security Alert   

      cluster regex_label  
0           0     Unknown  
1           1     Unknown  
2           2     Unknown  
3           0     Unknown  
4           0     Unknown  
...       ...         ...  
2405        0     Unknown  
2406        7     Unknown  
2407        0     Unknown  
2408        1     Unknown  
2409        7     Unknown  

[2410 rows x 6 columns]

In [29]:
df_non_regex= df[df['regex_label'] == "Unknown"]
df_non_regex

timestamp           source  \
0     2025-06-27 07:20:25        ModernCRM   
1         1/14/2025 23:07        ModernCRM   
2          1/17/2025 1:29  AnalyticsEngine   
3     2025-07-12 00:24:16         ModernHR   
4     2025-06-02 18:25:23    BillingSystem   
...                   ...              ...   
2405  2025-08-13 07:29:25         ModernHR   
2406       1/11/2025 5:32         ModernHR   
2407  2025-08-03 03:07:47    ThirdPartyAPI   
2408     11/11/2025 11:52    BillingSystem   
2409     12/25/2025 13:21  AnalyticsEngine   

                                            log_message    target_label  \
0     nova.osapi_compute.wsgi.server [req-b9718cd8-f...     HTTP Status   
1        Email service experiencing issues with sending  Critical Error   
2             Unauthorized access to data was attempted  Security Alert   
3     nova.osapi_compute.wsgi.server [req-4895c258-b...     HTTP Status   
4     nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...     HTTP Status   
...                                                 ...             ...   
2405  nova.osapi_compute.wsgi.server [req-96c3ec98-2...     HTTP Status   
2406  User 3844 account experienced multiple failed ...  Security Alert   
2407  nova.metadata.wsgi.server [req-b6d4a270-accb-4...     HTTP Status   
2408      Email service affected by failed transmission  Critical Error   
2409  Repeated failed login attempts occurred for us...  Security Alert   

      cluster regex_label  
0           0     Unknown  
1           1     Unknown  
2           2     Unknown  
3           0     Unknown  
4           0     Unknown  
...       ...         ...  
2405        0     Unknown  
2406        7     Unknown  
2407        0     Unknown  
2408        1     Unknown  
2409        7     Unknown  

[1910 rows x 6 columns]

In [42]:
df_non_regex[['target_label']].groupby(df_non_regex['source']).count()

target_label
source                       
AnalyticsEngine           371
BillingSystem             371
LegacyCRM                   7
ModernCRM                 373
ModernHR                  402
ThirdPartyAPI             386